In [1]:
# Root path of Fremont Dropbox
import os
import sys
import demand_util

# array analysis
import numpy as np
import osmnx as ox
import pandas as pd
import sklearn.cluster as skc
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

# geo spacial data analysis
import fiona
import geopandas as gpd
from shapely import wkt
from rtree import index
from keplergl import KeplerGl

# assorted parsing and modeling tools
import csv
import time
import math
import random
import requests
import alphashape
import multiprocessing 
from pytz import utc
from pyproj import Proj, transform
from shutil import copyfile, copytree
from sklearn.neighbors import BallTree
from matplotlib.path import Path as MPath
from here_util import shortest_path_by_travel_time
from shapely.ops import nearest_points, unary_union
from shapely.geometry import box, Point, LineString, Polygon, MultiPoint, MultiPolygon, GeometryCollection

# import polyline
from pathlib import Path

# importing all the Kepler.gl configurations
import ast

In [2]:
# updates modules when changed
%load_ext autoreload
%autoreload 2

In [3]:
project_delimitation = []
project_delimitation.append((-121.94277062699996, 37.55273259000006))
project_delimitation.append((-121.94099807399999, 37.554268507000074))
project_delimitation.append((-121.91790942699998, 37.549823434000075))
project_delimitation.append((-121.89348666299998, 37.52770136500004))
project_delimitation.append((-121.90056572499998, 37.52292299800007))
project_delimitation.append((-121.90817571699995, 37.52416183400004))
project_delimitation.append((-121.91252749099999, 37.51845069500007))
project_delimitation.append((-121.91349347899995, 37.513972023000065))
project_delimitation.append((-121.90855417099999, 37.503837324000074))
project_delimitation.append((-121.91358547299996, 37.50097863000008))
project_delimitation.append((-121.90798018999999, 37.49080413200005))
project_delimitation.append((-121.91894942199997, 37.48791568200005))
project_delimitation.append((-121.92029048799998, 37.488706567000065))
project_delimitation.append((-121.93070953799997, 37.48509600500006))
project_delimitation.append((-121.93254686299997, 37.48864173700008))
project_delimitation.append((-121.94079404499996, 37.50416395900004))
project_delimitation.append((-121.94569804899999, 37.51332606200003))
project_delimitation.append((-121.94918207899997, 37.520371545000046))
project_delimitation.append((-121.95305006999996, 37.52804520800004))
project_delimitation.append((-121.953966735, 37.53272020000003))
project_delimitation.append((-121.95428756799998, 37.53817435800005))
project_delimitation.append((-121.95506236799997, 37.54107322100003))
project_delimitation.append((-121.95676186899999, 37.54656695700004))
project_delimitation.append((-121.95529950799994, 37.54980786700003))
project_delimitation.append((-121.95261192399994, 37.550479763000055))
project_delimitation.append((-121.94988481799999, 37.55277211300006))
project_delimitation.append((-121.94613010599994, 37.55466923100005))
project_delimitation.append((-121.94277062699996, 37.55273259000006))

In [4]:
# Setting up the Coordinate Reference Systems up front in the necessary format.
crs_degree = {'init': 'epsg:4326'} # CGS_WGS_1984 (what the GPS uses)

# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from fremontdropbox import get_dropbox_location
# Root path of the Dropbox business account
dbx = get_dropbox_location()

# Temporary! Location of the folder where the restructuring is currently happening
data_path = dbx + '/Private Structured data collection'

In [5]:
sfcta_folder = os.path.join(data_path, "Data processing", "Raw", "Demand", "OD demand", "SFCTA demand data")
sections_path = os.path.join(data_path, "Aimsun", "Inputs", "sections.shp")
sections_shp = gpd.GeoDataFrame.from_file(sections_path)
sections_shp = sections_shp.to_crs(crs_degree) 

C:\Users\Theo\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [6]:
def from_csv(csv_df):
    all_points = []
    node_types = ['start', 'end']
    for node_type in node_types:
        points = list(zip(csv_df[node_type + '_node_lng'], csv_df[node_type + '_node_lat']))
        all_points.extend(points)
    return all_points

def to_csv(file_name, header, lines):
    def add_quotes(val):
        return "\"" + str(val) + "\"" if ',' in str(val) else str(val)

    csv = open(file_name, 'w')
    csv.write(header + '\n')
    for line in lines:
        csv.write(','.join(map(add_quotes, line)) + '\n')

In [109]:
def create_external_delim(dir_taz):
    ''' 
    Create a external demand delimitation:
    - load SFCTA data as Geopandas point (one point = one origin or one destination)
    - Get convex hull of the point
    - Use the convex hull (+ buffer) as the external demand delimitation
    '''
    # load the 3 csv files
    ending_csv = pd.read_csv(os.path.join(dir_taz, "ending_fremont_legs.csv"))
    internal_csv = pd.read_csv(os.path.join(dir_taz, "internal_fremont_legs.csv"))
    starting_csv = pd.read_csv(os.path.join(dir_taz, "starting_fremont_legs.csv"))

    # get the points from the csv's (start and end points)
    points = []
    points.extend(from_csv(ending_csv))
    points.extend(from_csv(internal_csv))
    points.extend(from_csv(starting_csv))
    points = np.array(points)

    # get convex hull of points
    hull = ConvexHull(points)
    hull_points = points[hull.vertices, :]

    # add buffer to convex hull
    def normalize(point):
        norm = np.linalg.norm(point)
        return point / norm if norm > 0 else point

    # for each point calculate the direction to expand for buffer
    buffer_directions = []
    for i in range(len(hull_points)):
        point = hull_points[i]
        left_neighbor = hull_points[(i-1) % len(hull_points)]
        right_neighbor = hull_points[(i+1) % len(hull_points)]
        left_arrow = point - left_neighbor
        right_arrow = point - right_neighbor
        left_arrow = normalize(left_arrow)
        right_arrow = normalize(right_arrow)
        buffer_directions.append(normalize(left_arrow + right_arrow))
    buffer_directions = np.array(buffer_directions)

    # calculate the new (expanded) hull points with buffer
    buffer_coefficient = .05
    expanded_hull_points = hull_points + buffer_coefficient * buffer_directions
    
    return expanded_hull_points

def create_external_centroids(sections_df, expanded_hull_points):
    '''
    Create external centroids:
    - select road with no fnode and capacity above 800 from sections_df
    - create a point at the end of all selected road
    - plot the points, get a list of points to remove visually
    '''
    # select roads with no fnode and capacity above 800 from sections_df
    sections_df = sections_df[pd.isnull(sections_df['fnode']) & (sections_df['capacity'] > 800)]
    sections_df = sections_df[['eid', 'geometry']]

    # filter out roads that are visually erroneous -> a road not entering the project area (Fremont)
    # sections_df.to_csv('selected_roads.csv')   # roads to remove obtained visually
    ######################################################################################
    ########################## NEED AN AUTOMATED WAY TO DO THIS ##########################
    ######################################################################################
    roads_to_remove = [56744, 30676, 35572, 56534]
    sections_df = sections_df.astype({'eid': 'int32'})
    sections_df = sections_df[~sections_df['eid'].isin(roads_to_remove)]

    # create external centroid nodes -> create a point at the terminal end of these roads
    # that is, for each road find the end of the road that is closer to the external delimitation (convex hull)
    external_centroid_nodes = []
    internal_centroid_nodes = []  # need later to compute center point of project area
    circle = np.concatenate((expanded_hull_points, expanded_hull_points[0][None, :]), axis=0)
    external_delimitation = LineString(circle)
    for road in sections_df['geometry']:
        start_point = Point(road.coords[0])
        end_point = Point(road.coords[-1])

        if external_delimitation.distance(start_point) < external_delimitation.distance(end_point):
            # start is external centroid
            external_centroid_nodes.append(start_point)
            internal_centroid_nodes.append(end_point)
        else:
            # end is external centroid
            external_centroid_nodes.append(end_point)
            internal_centroid_nodes.append(start_point)
            
    return external_centroid_nodes, internal_centroid_nodes

def create_mesh(expanded_hull_points, project_delimitation, mesh_density=0.01, dense_portion=0.25, return_map=False):
    ''' 
    Create mesh of points.
    
    Note: mesh_density 0.001 creates 2 million points
    '''
    external_delimitation_poly = Polygon(expanded_hull_points)
    project_delimitation_poly = Polygon(project_delimitation)
    external_minus_project = external_delimitation_poly.difference(project_delimitation_poly)
    proj_center = project_delimitation_poly.centroid
    
    x_min, x_max = np.min(expanded_hull_points[:, 0]), np.max(expanded_hull_points[:, 0])
    y_min, y_max = np.min(expanded_hull_points[:, 1]), np.max(expanded_hull_points[:, 1])
    x, y = np.meshgrid(np.arange(x_min, x_max, mesh_density), np.arange(y_min, y_max, mesh_density))
    points = np.vstack((x.ravel(), y.ravel())).T
    
    cx_min, cx_max = proj_center.x - ((x_max - x_min) * dense_portion), proj_center.x + ((x_max - x_min) * dense_portion)
    cy_min, cy_max = proj_center.y - ((y_max - y_min) * dense_portion), proj_center.y + ((y_max - y_min) * dense_portion)
    cx, cy = np.meshgrid(np.arange(cx_min, cx_max, mesh_density / 4), np.arange(cy_min, cy_max, mesh_density / 4))
    cpoints = np.vstack((cx.ravel(), cy.ravel())).T
    
    points = np.unique(np.concatenate([points, cpoints], axis=0), axis=0)
    
    ext_bounds = np.asarray(list(zip(*external_minus_project.exterior.coords.xy)))
    int_bounds = np.asarray(list(zip(*external_minus_project.interiors[0].coords.xy)))
    ext_path, int_path = MPath(ext_bounds), MPath(int_bounds)
    
    ext_mask = ext_path.contains_points(points)
    int_mask = np.invert(int_path.contains_points(points))
    mask = np.bitwise_and(ext_mask, int_mask)
    
    mesh_points = points[mask]
    
    if return_map:
        mesh_map = KeplerGl(height=600)
        mesh_map.add_data(data=gpd.GeoDataFrame(geometry=gpd.points_from_xy(mesh_points[:, 0], mesh_points[:, 1])))
        return mesh_map
                          
    return mesh_points

def clean_ext_taz(external_taz, project_delimitation):
    '''
    Clean and process generated External TAZ polygons.
    '''
    for idx, hull in external_taz.iterrows():
        if type(hull['geometry']) != Polygon:
            bounds = Polygon(max(hull['geometry'], key=lambda a: a.area).exterior)
            secondary = min(hull['geometry'], key=lambda a: a.area)
            if secondary.area > 0.001:
                nearest_idx = min([i for i in range(len(external_taz.index)) if i != idx], \
                                   key=lambda i: external_taz.loc[i, 'geometry'].distance(secondary))
                nearest = external_taz.loc[nearest_idx, 'geometry']
                external_taz.loc[nearest_idx, 'geometry'] = nearest.union(secondary)
        else:
            bounds = hull['geometry']
        external_taz.loc[idx, 'geometry'] = bounds

    for i in range(len(external_taz.index)):
        for j in range(len(external_taz.index)):
            hull_i, hull_j = external_taz.loc[i, 'geometry'], external_taz.loc[j, 'geometry']
            if hull_i.intersects(hull_j):
                hull_diff = hull_i.intersection(hull_j)
                max_hull = max([hull_i, hull_j], key=lambda a: a.area)

                if max_hull == hull_i:
                    external_taz.loc[i, 'geometry'] = hull_i.difference(hull_diff)
                    external_taz.loc[j, 'geometry'] = hull_j.union(hull_diff)
                else:
                    external_taz.loc[i, 'geometry'] = hull_i.union(hull_diff)
                    external_taz.loc[j, 'geometry'] = hull_j.difference(hull_diff)     

    for idx, hull in external_taz.iterrows():
        if type(hull['geometry']) != Polygon:
            bounds = Polygon(max(hull['geometry'], key=lambda a: a.area).exterior)
            secondary = min(hull['geometry'], key=lambda a: a.area)
            nearest_idx = min([i for i in range(len(external_taz.index)) if i != idx], \
                               key=lambda i: external_taz.loc[i, 'geometry'].distance(secondary))
            nearest = external_taz.loc[nearest_idx, 'geometry']
            external_taz.loc[nearest_idx, 'geometry'] = nearest.union(secondary)
        else:
            bounds = hull['geometry']
        external_taz.loc[idx, 'geometry'] = bounds

    for idx, hull in external_taz.iterrows():
        external_taz.loc[idx, 'geometry'] = external_taz.loc[idx, 'geometry'].difference(Polygon(project_delimitation))
    
    return external_taz

def create_external_taz(dir_taz, sections_df, project_delimitation,\
                        mesh_density=0.01, output_dir=None, return_with_map=False):
    """
    3 Steps for Create external TAZs
    1. Create a external demand delimitation:
    - load SFCTA data as Geopandas point (one point = one origin or one destination)
    - Get convex hull of the point
    - Use the convex hull (+ buffer) as the external demand delimitation
    2. create external centroids:
    - select road with no fnode and capacity above 800 from sections_df
    - create a point at the end of all selected road
    - plot the points, get a list of points to remove visually
    3. create external TAZs:
    - create a mesh a points inside the external demand delimitation and outside the internal demand delimitation (project delimitation)
    - use a Direction API (maybe Here direction):
    for every mesh point:
        Query path from mesh point to center of the project area
        Find the closest external centroid to the path. Test that all paths are not to far from existing
            external centroid --> if not, we might be missing one external centroid.
        Associate the external centroid to the mesh point.
        create external TAZ from mesh of points (if you reach point, Theo has already done it for internal TAZs)

    @param dir_taz:         folder containing prefix_fremont_legs.csv where prefix=ending, internal and starting
    @param sections_df:     geo pandas data frame of the aimsun sections
    """
    # 1. Create a external demand delimitation:
    expanded_hull_points = create_external_delim(dir_taz)
    # 2. create external centroids:
    external_centroid_nodes, internal_centroid_nodes = create_external_centroids(sections_df, expanded_hull_points)
    # 3. create external TAZs:
    
    #create mesh
    mesh_points = create_mesh(expanded_hull_points, project_delimitation, mesh_density=mesh_density, dense_portion=0.1,\
                              return_map=False)
    
    # create balltree from mesh points
    mesh_df = pd.DataFrame()
    mesh_df["lon"], mesh_df["lat"] = mesh_points[:,0], mesh_points[:,1]
    mesh_df['class'] = -1  
    mesh_size = len(mesh_df.index)
    mesh_bt = BallTree(np.deg2rad(mesh_df[['lat', 'lon']].values), metric='haversine')
    
    # compute center of project area
    internal_centroid_nodes = np.array([(p.x, p.y) for p in internal_centroid_nodes])
    project_center = np.mean(internal_centroid_nodes, axis=0)
    project_center = Point(project_center[0], project_center[1])

    # for each mesh point find closest external centroid to its query path (parallelized classification)
    # BOTTLENECK. COMPLETION TIME INVERSELY RELATED TO NUM_CORES ON MACHINE.
    project_delimitation_line = LineString(project_delimitation + [project_delimitation[0]])   
    
    print("Starting classification, {} points in total...".format(mesh_size))
 
    pool = multiprocessing.Pool() 

    start = time.time()
    result_async = [(row, pool.apply_async(shortest_path_by_travel_time,\
                                     args = (Point(row['lon'], row['lat']), project_center)))\
                    for index, row in mesh_df.iterrows()] 
    paths = [(row, res.get()) for row, res in result_async] 
    print("Computed paths from points, took {} in total.".format(time.time() - start))
    
    pool.close()
    
    del pool, result_async
    
    for row, path in paths:
        if not path:
            query_point = np.deg2rad(np.asarray([[row['lat'], row['lon']]]))
            distances, indices = mesh_bt.query(query_point, k=1)
            if mesh_df.loc[list(indices[0])[0]]['class'] != -1:
                mesh_df.loc[(mesh_df['lon'] == row['lon']) & (mesh_df['lat'] == row['lat']),\
                            ['class']] = mesh_df.loc[list(indices[0])[0]]['class']
                continue
            else:
                path = LineString([Point(row['lon'], row['lat']), project_center])
                
        intersect_point = project_delimitation_line.intersection(path)

        # find closest centroid to intersection point
        min_distance, closest_centroid, best_classification = float('inf'), None, None
        for idx, centroid in enumerate(external_centroid_nodes):
            dist = intersect_point.distance(centroid)
            if dist < min_distance:
                min_distance = dist
                closest_centroid = centroid
                best_classification = idx
        
        mesh_df.loc[(mesh_df['lon'] == row['lon']) & (mesh_df['lat'] == row['lat']), ['class']] = best_classification
    
    # create TAZs from classified mesh
    mesh_squares = gpd.GeoDataFrame(data=mesh_df, geometry=gpd.points_from_xy(mesh_df.lon, mesh_df.lat))
    mesh_squares.geometry = mesh_squares.buffer(mesh_density/1.99).envelope
    external_taz = mesh_squares[['class', 'geometry']].dissolve(by='class')
    external_taz = clean_ext_taz(external_taz, project_delimitation)
    
    if return_with_map:
        ext_gdf = gpd.GeoDataFrame(geometry=external_centroid_nodes)
        int_gdf = gpd.GeoDataFrame(geometry=internal_centroid_nodes)
        mesh_data = gpd.GeoDataFrame(data=mesh_df, \
                                     geometry=gpd.points_from_xy(mesh_df.lon, mesh_df.lat))[['class', 'geometry']]
        
        ext_taz_map = KeplerGl(height=1000)
        ext_taz_map.add_data(data=external_taz, name="External TAZs")
        ext_taz_map.add_data(data=ext_gdf, name = "External Centroids")
        ext_taz_map.add_data(data=int_gdf, name = "Internal Centroids")
        ext_taz_map.add_data(data=mesh_data, name = "Classification Set")
        
        return external_taz, ext_taz_map
    
    return external_taz

In [ ]:
sfcta_folder = os.path.join(data_path, "Data processing", "Raw", "Demand", "OD demand", "SFCTA demand data")
sections_path = os.path.join(data_path, "Aimsun", "Inputs", "sections.shp")

sections_shp = gpd.GeoDataFrame.from_file(sections_path)
sections_shp = sections_shp.to_crs(crs_degree) 
# print(sections_shp.columns)
# print(sections_shp.head)
    
if __name__ == '__main__':
    external_taz, ext_taz_map = create_external_taz(sfcta_folder, sections_shp, project_delimitation, return_with_map=True)

In [112]:
external_taz.to_file("external_taz.shp")

In [ ]:
def convert_point_to_coord(string):
    lon, lat = string.split(' ')[1:]
    lon = float(lon.split('(')[1])
    lat = float(lat.split(')')[0])
    return [lon, lat]

In [ ]:
def render_taz_from_csv(csv_file, output_dir=None):
    project_delimitation_line = LineString(project_delimitation + [project_delimitation[0]])
    
    info_points_df = pd.read_csv(csv_file)
    external_centroid_nodes = info_points_df['closest_external_centroid'].map(convert_point_to_coord).tolist()
    external_centroid_nodes = [Point(coord[0], coord[1]) for coord in external_centroid_nodes]
    
    kepler_map = KeplerGl(height=600)
    #kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': [project_center]}, crs='epsg:4326'), name='project_center')
    kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': external_centroid_nodes}, crs='epsg:4326'), name='external_centroids')
    kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': [project_delimitation_line]}, crs='epsg:4326'), name='project_delimitation')
    #kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': [external_delimitation]}, crs='epsg:4326'), name='external_delimitation')
    
    taz_id = 0
    boundary_list = []
    taz_name_list = []
    external_centroids = info_points_df['closest_external_centroid'].unique()
    for centroid in external_centroids:
        taz_df = info_points_df.loc[info_points_df['closest_external_centroid']==centroid]
        taz_points = np.array(taz_df['origin_mesh_point'].map(convert_point_to_coord).tolist())
        taz_hull = ConvexHull(taz_points)
        taz_boundary = taz_points[taz_hull.vertices, :]
        taz_poly = Polygon(taz_boundary)
        taz_name = 'External TAZ' + str(taz_id)
        kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': taz_poly}, crs='epsg:4326',index=[0]), name=taz_name)
        taz_name_list.append(taz_name)
        boundary_list.append(taz_poly)
        taz_id += 1
    taz_gpd = gpd.GeoDataFrame({'taz_name':taz_name_list,'geometry':boundary_list})
    file_path = 'external_taz.html'
    if output_dir:
        file_path = os.path.join(output_dir, file_path)
    kepler_map.save_to_html(file_name=file_path)
    return taz_gpd

In [ ]:
# print(sections_shp.columns)
# print(sections_shp.head)
output_dir = os.path.join(data_path, 'Data processing', 'Kepler maps', 'HereAPI')
mesh_points_to_centroid_file_path = 'mesh_point_to_centroid.csv'
taz_gpd = render_taz_from_csv(os.path.join(output_dir, mesh_points_to_centroid_file_path), output_dir=output_dir)

## To do:
1. Split the function create_external_taz into several subfunction that needs to be run sequentially
    1. Create a external demand delimitation:
    2. Create external centroids:
    3. Create external TAZs:
        1. Create a mesh grid
        2. Associate every mesh grid to an external centroid
        3. From the mesh grid create the external TAZ (Theo can do it)
2. For every sub-function write some code to check the function (or render the output of the function)